In [1]:

import numpy as np

from entities.entity import Entity
from entities.animal import Animal 
from entities.microphone import MicrophoneStation

SPEED_OF_SOUND = 343 # m/s

In [2]:
# the following function generated using GPT-4
def trilaterate(p1, r1, p2, r2, p3, r3):
    # Calculate relative positions of point 2 and point 3
    ex = (p2 - p1) / np.linalg.norm(p2 - p1)
    i = np.dot(ex, p3 - p1)
    ey = (p3 - p1 - i * ex) / np.linalg.norm(p3 - p1 - i * ex)
    ez = np.cross(ex, ey)

    # Calculate the distances
    d = np.linalg.norm(p2 - p1)
    j = np.dot(ey, p3 - p1)

    # Calculate the position of the intersection point
    x = (r1 ** 2 - r2 ** 2 + d ** 2) / (2 * d)
    y = (r1 ** 2 - r3 ** 2 + i ** 2 + j ** 2) / (2 * j) - (i / j) * x

    # Calculate the z-coordinate, if possible
    z_squared = r1 ** 2 - x ** 2 - y ** 2
    if z_squared < 0:
        return None  # No intersection
    z = np.sqrt(z_squared)

    # Calculate the coordinates
    p = p1 + x * ex + y * ey + z * ez
    return p

In [3]:
# Set an animal true location
truth_animal = Animal()
truth_animal.setLLA((-48.0,134.0,10.0))
print(f'True animal LLA', truth_animal.getLLA())

# set the location of three sensors around this animal
test_sensor_1 = MicrophoneStation(lla=(-48.0,133.99,10.0))
print(f'Sensor 1    LLA', test_sensor_1.getLLA())

test_sensor_2 = MicrophoneStation(lla=(-48.0,134.05,10.0))
print(f'Sensor 2    LLA', test_sensor_2.getLLA())

test_sensor_3 = MicrophoneStation(lla=(-48.01,134.0,10.0))
print(f'Sensor 3    LLA', test_sensor_3.getLLA())

test_sensor_4 = MicrophoneStation(lla=(-48.01,134.04,10.0))
print(f'Sensor 4    LLA', test_sensor_3.getLLA())

print("distance 1", test_sensor_1.distance(truth_animal))
print("distance 2", test_sensor_2.distance(truth_animal))
print("distance 3", test_sensor_3.distance(truth_animal))
print("distance 4", test_sensor_4.distance(truth_animal))

# Sphere centers and radii and travel times
p1 = np.array(test_sensor_1.getECEF())
r1 = test_sensor_1.distance(truth_animal)
sound_travel_time_1 = r1 / SPEED_OF_SOUND

p2 = np.array(test_sensor_2.getECEF())
r2 = test_sensor_2.distance(truth_animal)
sound_travel_time_2 = r2 / SPEED_OF_SOUND

p3 = np.array(test_sensor_3.getECEF())
r3 = test_sensor_3.distance(truth_animal)
sound_travel_time_3 = r3 / SPEED_OF_SOUND

p4 = np.array(test_sensor_4.getECEF())
r4 = test_sensor_4.distance(truth_animal)
sound_travel_time_4 = r4 / SPEED_OF_SOUND

print("sound travel time 1", sound_travel_time_1)
print("sound travel time 2", sound_travel_time_2)
print("sound travel time 3", sound_travel_time_3)
print("sound travel time 4", sound_travel_time_4)

# Determine the closest sensor i.e. with smallest travel time and set time to zero
min_time = np.min([sound_travel_time_1,sound_travel_time_2,sound_travel_time_3,sound_travel_time_4])

# Determine the observed trigger times
st_obs_1 = sound_travel_time_1 - min_time
st_obs_2 = sound_travel_time_2 - min_time
st_obs_3 = sound_travel_time_3 - min_time
st_obs_4 = sound_travel_time_4 - min_time

print("sound trigger time 1", st_obs_1)
print("sound trigger time 2", st_obs_2)
print("sound trigger time 3", st_obs_3)
print("sound trigger time 4", st_obs_4)

# Trilateration
intersection = trilaterate(p1, r1, p2, r2, p3, r3)
if intersection is not None:
    print(f"Intersection point ECEF: {intersection}")
    predicted_animal = Animal()
    predicted_animal.setECEF(intersection)
    print(f'Predicted animal LLA', predicted_animal.getLLA())
else:
    print("No intersection found.")

True animal LLA (-48.0, 134.0, 10.0)
Sensor 1    LLA (-48.0, 133.99, 10.0)
Sensor 2    LLA (-48.0, 134.05, 10.0)
Sensor 3    LLA (-48.01, 134.0, 10.0)
Sensor 4    LLA (-48.01, 134.0, 10.0)
distance 1 746.2547025302385
distance 2 3731.2733989899125
distance 3 1111.9059424847765
distance 4 3185.1137618235216
sound travel time 1 2.17566968667708
sound travel time 2 10.878348102011406
sound travel time 3 3.2417082871276284
sound travel time 4 9.286045952838256
sound trigger time 1 0.0
sound trigger time 2 8.702678415334326
sound trigger time 3 1.0660386004505482
sound trigger time 4 7.110376266161175
Intersection point ECEF: [-2970167.60763061  3075698.59488686 -4716883.43772499]
Predicted animal LLA (-48.00000000042609, 133.99999999863581, 9.5641894293949)


In [4]:
def solve_animal_lla(sensor_1,sensor_2,sensor_3,sensor_4,st_est_1,st_est_2,st_est_3,st_est_4):
    # grid search
    estimate_animal = Animal()
    best_err = 100000000

    for lat in np.arange(-48.1,-47.9,0.01):
        for lon in np.arange(133.9,134.1,0.01):
            
            # guess it is here...
            estimate_animal.setLLA((lat, lon, 10.0))
            
            # so calculate the times given that guess
            r1 = sensor_1.distance(estimate_animal)
            s_1 = r1 / SPEED_OF_SOUND

            r2 = sensor_2.distance(estimate_animal)
            s_2 = r2 / SPEED_OF_SOUND

            r3 = sensor_3.distance(estimate_animal)
            s_3 = r3 / SPEED_OF_SOUND
            
            r4 = sensor_4.distance(estimate_animal)
            s_4 = r4 / SPEED_OF_SOUND        
            
            # Determine the closest sensor i.e. with smallest travel time and set time to zero
            min_time = np.min([s_1,s_2,s_3,s_4])

            # Determine the trigger times
            st_est_1 = s_1 - min_time
            st_est_2 = s_2 - min_time
            st_est_3 = s_3 - min_time
            st_est_4 = s_4 - min_time       
            
            # calculate the error with times observed
            err = abs(st_obs_1-st_est_1) + \
                  abs(st_obs_2-st_est_2) + \
                  abs(st_obs_3-st_est_3) + \
                  abs(st_obs_4-st_est_4) 
                
            if err < best_err:
                best_err = err
                best_lat,best_lon=lat,lon
                print(f'New best lat: {lat} lon: {lon} absolute error in arrival times: {err}')
                
    return best_lat,best_lon
                
# using only the information about sensors and observed trigger times
lat,lon = solve_animal_lla(
    test_sensor_1,test_sensor_2,test_sensor_3,test_sensor_4,
    st_obs_1,st_obs_2,st_obs_3,st_obs_4)  

print(f'Estimated Animal lat: {lat} lon: {lon}')
    

New best lat: -48.1 lon: 133.9 absolute error in arrival times: 4.777433396892862
New best lat: -48.09 lon: 133.9 absolute error in arrival times: 4.535763480085771
New best lat: -48.080000000000005 lon: 133.9 absolute error in arrival times: 4.2397024151799325
New best lat: -48.07000000000001 lon: 133.9 absolute error in arrival times: 3.8774259423875157
New best lat: -48.06000000000001 lon: 133.91 absolute error in arrival times: 3.8274754506041346
New best lat: -48.00000000000002 lon: 133.99999999999991 absolute error in arrival times: 1.0076472989339891e-10
Estimated Animal lat: -48.00000000000002 lon: 133.99999999999991
